In [3]:
import pandas as pd
import numpy as np

Reading New York data-- i have downloaded the data into csv file and added to watson studio

In [13]:
# The code was removed by Watson Studio for sharing.

,ZIP,Lat,Long
0,10001,40.750633,-73.997177
1,10002,40.715775,-73.986212
2,10003,40.731829,-73.989181
3,10004,40.688630,-74.018244
4,10005,40.706027,-74.008835


In [38]:

body = client_9931a902e795490ea510bfbf76e35bf1.get_object(Bucket='myassignment-donotdelete-pr-xoiu6dbqezzdve',Key='Location1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_4 = pd.read_csv(body)
df_data_4.head()

body = client_9931a902e795490ea510bfbf76e35bf1.get_object(Bucket='myassignment-donotdelete-pr-xoiu6dbqezzdve',Key='popdata1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_ny_pop = pd.read_csv(body, thousands=",")
df_ny_pop.head()
#df_ny_pop.shape

,ZIP,Population
0,10001,21102
1,10002,81410
2,10003,56024
3,10004,3089
4,10005,7135


In [39]:
df_ny_info=pd.merge(df_ny,df_ny_pop,on='ZIP')

In [40]:
#df_ny_info.head(5)
print(df_ny_info.columns)

Index(['ZIP', 'Lat', ' Long', 'Population'], dtype='object')


In [85]:
import folium
latitude=40.7999209
longitude=-73.9683102
map_NY = folium.Map(location=[latitude, longitude], zoom_start=11)
map_NY

# adding markers to map
for lat, lng, Population, ZIP in zip(df_ny_info['Lat'], df_ny_info[' Long'],df_ny_info['Population'], df_ny_info['ZIP']):
    label = '{}, {}'.format(ZIP, Population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)
    
map_NY

In [83]:
limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'O0S5BRBBXQ3IMJZSZIFMVS2T31JXLRDFKBYJ3QXRL0OUCYOD'
CLIENT_SECRET = 'TJDWYDKOCFDH33UXN2ZQHWOBW05RZWSFMJ13QRHMJL03AOS0'
VERSION = '20190530'

In [84]:
# function to repeat the exploring process to all the neighborhoods in Newyork
import requests
LIMIT=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
      #  print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
       
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
        v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                            'Venue Id']
    
    return(nearby_venues)

In [45]:
ny_venues = getNearbyVenues(names=df_ny_info['ZIP'],
                                   latitudes=df_ny_info['Lat'],
                                   longitudes=df_ny_info[' Long']
                                  )

In [98]:
ny_venues
ny_restaurant=ny_venues[ny_venues['Venue Category'].str.contains('Restaurant')|ny_venues['Venue Category'].str.contains('Food')|ny_venues['Venue Category'].str.contains('Market')]
ny_restaurant.shape

(1216, 8)

In [99]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_restaurant[['Venue Category']], prefix="", prefix_sep="")


ny_onehot['Neighborhood'] = ny_restaurant['Neighborhood'] 


fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,...,Szechuan Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
3,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [100]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()


In [121]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

#ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [123]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(50)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,10001,Chinese Restaurant,Restaurant,Food Court,Italian Restaurant,French Restaurant
1,10002,Mexican Restaurant,American Restaurant,Chinese Restaurant,Vietnamese Restaurant,French Restaurant
2,10003,Vegetarian / Vegan Restaurant,Japanese Restaurant,Sushi Restaurant,Mediterranean Restaurant,Chinese Restaurant
3,10004,Food Truck,Mexican Restaurant,Seafood Restaurant,Food Stand,Food Court
4,10005,American Restaurant,Italian Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant
5,10006,Food Truck,Italian Restaurant,Japanese Curry Restaurant,Falafel Restaurant,Fast Food Restaurant
6,10007,French Restaurant,American Restaurant,Italian Restaurant,Sushi Restaurant,Falafel Restaurant
7,10009,Mexican Restaurant,Italian Restaurant,Latin American Restaurant,American Restaurant,Vegetarian / Vegan Restaurant
8,10010,Indian Restaurant,Italian Restaurant,Mexican Restaurant,American Restaurant,Restaurant
9,10011,American Restaurant,Seafood Restaurant,Tapas Restaurant,Italian Restaurant,Japanese Curry Restaurant


In [124]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [125]:
df_pop=df_ny_info.copy()
df_pop.head()

,ZIP,Lat,Long,Population
0,10001,40.750633,-73.997177,21102
1,10002,40.715775,-73.986212,81410
2,10003,40.731829,-73.989181,56024
3,10004,40.688630,-74.018244,3089
4,10005,40.706027,-74.008835,7135


In [126]:
df_pop=df_pop.rename(columns={"ZIP":"Neighborhood"})
df_pop.head()

,Neighborhood,Lat,Long,Population
0,10001,40.750633,-73.997177,21102
1,10002,40.715775,-73.986212,81410
2,10003,40.731829,-73.989181,56024
3,10004,40.688630,-74.018244,3089
4,10005,40.706027,-74.008835,7135


In [127]:
neighborhoods_venues_sorted=pd.merge(neighborhoods_venues_sorted,df_pop,on='Neighborhood')

In [128]:
neighborhoods_venues_sorted.head(5)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Lat,Long,Population
0,0,10001,Chinese Restaurant,Restaurant,Food Court,Italian Restaurant,French Restaurant,40.750633,-73.997177,21102
1,0,10002,Mexican Restaurant,American Restaurant,Chinese Restaurant,Vietnamese Restaurant,French Restaurant,40.715775,-73.986212,81410
2,0,10003,Vegetarian / Vegan Restaurant,Japanese Restaurant,Sushi Restaurant,Mediterranean Restaurant,Chinese Restaurant,40.731829,-73.989181,56024
3,0,10004,Food Truck,Mexican Restaurant,Seafood Restaurant,Food Stand,Food Court,40.688630,-74.018244,3089
4,0,10005,American Restaurant,Italian Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,40.706027,-74.008835,7135


In [129]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Lat'], neighborhoods_venues_sorted[' Long'], neighborhoods_venues_sorted['Neighborhood'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
#map_clusters.save("clustered.html")

In [141]:
#neighborhoods_venues_.Population = neighborhoods_venues_.Population.astype(float)
#neighborhoods_venues_sorted = neighborhoods_venues_sorted.sort_values(by='Population',ascending=False)
#df[df['count'] == df.groupby('name')['count'].transform('max')]
#neighbourhoods_grouped_top_pop = neighborhoods_venues_sorted.groupby('Cluster Labels')['Population'].transform(max)
neighbourhoods_grouped_top_pop = neighborhoods_venues_sorted[neighborhoods_venues_sorted['Population']==neighborhoods_venues_sorted.groupby('Cluster Labels')['Population'].transform(max)]
#neighbourhoods_grouped_top_pop.head(5)
neighbourhoods_grouped_top_pop.head()
#neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Lat,Long,Population
22,0,10025,Indian Restaurant,Mexican Restaurant,Chinese Restaurant,Thai Restaurant,Szechuan Restaurant,40.798601,-73.966622,94600
26,2,10029,Mexican Restaurant,Thai Restaurant,Latin American Restaurant,Spanish Restaurant,Restaurant,40.791763,-73.943970,76003
48,1,10128,Mexican Restaurant,Vietnamese Restaurant,Indian Restaurant,Japanese Restaurant,Chinese Restaurant,40.781432,-73.950013,60453


In [145]:
import folium
latitude=40.7128
longitude=-74.0060
map_NY = folium.Map(location=[latitude, longitude], zoom_start=10)
map_NY

# adding markers to map
for lat,lng, neigh,clabel in zip(neighbourhoods_grouped_top_pop['Lat'],neighbourhoods_grouped_top_pop[' Long'],neighbourhoods_grouped_top_pop['Neighborhood'],neighbourhoods_grouped_top_pop['Cluster Labels']):
    label = '{},{}'. format(neigh,clabel)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  

map_NY